## Prepare, clean and split Data for loading into Postgress Tables

In [1]:
# Import dependencies
import json 
import pandas as pd

import psycopg2
from sqlalchemy import create_engine
from config import user_key

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Read the movies.json for financial information
df_movies = pd.read_json("Data/movies.json")
df_movies.head()

,Title,US Gross,Worldwide Gross,US DVD Sales,Production Budget,Release Date,MPAA Rating,Running Time min,Distributor,Source,Major Genre,Creative Type,Director,Rotten Tomatoes Rating,IMDB Rating,IMDB Votes
0,The Land Girls,146083.0,146083.0,NaN,8000000.0,Jun 12 1998,R,NaN,Gramercy,None,None,None,None,NaN,6.1,1071.0
1,"First Love, Last Rites",10876.0,10876.0,NaN,300000.0,Aug 07 1998,R,NaN,Strand,None,Drama,None,None,NaN,6.9,207.0
2,I Married a Strange Person,203134.0,203134.0,NaN,250000.0,Aug 28 1998,None,NaN,Lionsgate,None,Comedy,None,None,NaN,6.8,865.0
3,Let's Talk About Sex,373615.0,373615.0,NaN,300000.0,Sep 11 1998,None,NaN,Fine Line,None,Comedy,None,None,13.0,NaN,NaN
4,Slam,1009819.0,1087521.0,NaN,1000000.0,Oct 09 1998,R,NaN,Trimark,Original Screenplay,Drama,Contemporary Fiction,None,62.0,3.4,165.0


In [4]:
# Read the oscars_df.csv for oscar and movie information
df_oscar = pd.read_csv("Data/oscars_df.csv", index_col=0)
df_oscar.head()

,Film,Oscar Year,Film Studio/Producer(s),Award,Year of Release,Movie Time,Movie Genre,IMDB Rating,IMDB Votes,Movie Info,...,Tomatometer Status,Tomatometer Rating,Tomatometer Count,Audience Status,Audience Rating,Audience Count,Tomatometer Top Critics Count,Tomatometer Fresh Critics Count,Tomatometer Rotten Critics Count,Film ID
0,Wings,1927/28,Famous Players-Lasky,Winner,1927,144,"Drama,Romance,War",7.5,"12,221","With World War I afoot, David Armstrong (Richa...",...,Certified-Fresh,93.0,46.0,Upright,78.0,3530.0,9.0,43.0,3.0,2becf7d5-a3de-46ab-ae45-abdd6b588067
1,7th Heaven,1927/28,Fox,Nominee,1927,110,"Drama,Romance",7.7,"3,439",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19ed3295-a878-4fd2-8e60-5cd7b5f93dad
2,The Racket,1927/28,The Caddo Company,Nominee,1928,84,"Crime,Drama,Film-Noir",6.7,"1,257",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3111c2d8-0908-4093-8ff3-99c89f2f2f08
3,The Broadway Melody,1928/29,Metro-Goldwyn-Mayer,Winner,1929,100,"Drama,Musical,Romance",5.7,"6,890","Vaudeville sisters ""Hank"" (Bessie Love) and Qu...",...,Rotten,33.0,24.0,Spilled,21.0,1813.0,7.0,8.0,16.0,de063f3f-2d35-4e1c-8636-6eb4c16bd236
4,Alibi,1928/29,Feature Productions,Nominee,1929,91,"Action,Crime,Romance",5.8,765,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,609887c2-877c-43a4-b88c-e40e31096a98


In [5]:
# Generate the movie id as the first column
df_oscar.insert(loc=0, 
               column='movie_id',
               value=(df_oscar.index + 1))
df_oscar.head()

,movie_id,Film,Oscar Year,Film Studio/Producer(s),Award,Year of Release,Movie Time,Movie Genre,IMDB Rating,IMDB Votes,...,Tomatometer Status,Tomatometer Rating,Tomatometer Count,Audience Status,Audience Rating,Audience Count,Tomatometer Top Critics Count,Tomatometer Fresh Critics Count,Tomatometer Rotten Critics Count,Film ID
0,1,Wings,1927/28,Famous Players-Lasky,Winner,1927,144,"Drama,Romance,War",7.5,"12,221",...,Certified-Fresh,93.0,46.0,Upright,78.0,3530.0,9.0,43.0,3.0,2becf7d5-a3de-46ab-ae45-abdd6b588067
1,2,7th Heaven,1927/28,Fox,Nominee,1927,110,"Drama,Romance",7.7,"3,439",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19ed3295-a878-4fd2-8e60-5cd7b5f93dad
2,3,The Racket,1927/28,The Caddo Company,Nominee,1928,84,"Crime,Drama,Film-Noir",6.7,"1,257",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3111c2d8-0908-4093-8ff3-99c89f2f2f08
3,4,The Broadway Melody,1928/29,Metro-Goldwyn-Mayer,Winner,1929,100,"Drama,Musical,Romance",5.7,"6,890",...,Rotten,33.0,24.0,Spilled,21.0,1813.0,7.0,8.0,16.0,de063f3f-2d35-4e1c-8636-6eb4c16bd236
4,5,Alibi,1928/29,Feature Productions,Nominee,1929,91,"Action,Crime,Romance",5.8,765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,609887c2-877c-43a4-b88c-e40e31096a98


In [6]:
# Merge oscar data with the movies data
df_joined =pd.merge(df_oscar, df_movies, how='left', left_on='Film', right_on='Title')
df_joined.head()

,movie_id,Film,Oscar Year,Film Studio/Producer(s),Award,Year of Release,Movie Time,Movie Genre,IMDB Rating_x,IMDB Votes_x,...,MPAA Rating,Running Time min,Distributor,Source,Major Genre,Creative Type,Director,Rotten Tomatoes Rating,IMDB Rating_y,IMDB Votes_y
0,1,Wings,1927/28,Famous Players-Lasky,Winner,1927,144,"Drama,Romance,War",7.5,"12,221",...,None,NaN,None,None,None,None,None,96.0,7.9,3035.0
1,2,7th Heaven,1927/28,Fox,Nominee,1927,110,"Drama,Romance",7.7,"3,439",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,The Racket,1927/28,The Caddo Company,Nominee,1928,84,"Crime,Drama,Film-Noir",6.7,"1,257",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,The Broadway Melody,1928/29,Metro-Goldwyn-Mayer,Winner,1929,100,"Drama,Musical,Romance",5.7,"6,890",...,None,NaN,MGM,Original Screenplay,Musical,None,None,38.0,6.7,2017.0
4,5,Alibi,1928/29,Feature Productions,Nominee,1929,91,"Action,Crime,Romance",5.8,765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Drop the duplicated rows
df_joined.drop_duplicates(subset='movie_id', keep='first', inplace=True)
df_joined.head()

,movie_id,Film,Oscar Year,Film Studio/Producer(s),Award,Year of Release,Movie Time,Movie Genre,IMDB Rating_x,IMDB Votes_x,...,MPAA Rating,Running Time min,Distributor,Source,Major Genre,Creative Type,Director,Rotten Tomatoes Rating,IMDB Rating_y,IMDB Votes_y
0,1,Wings,1927/28,Famous Players-Lasky,Winner,1927,144,"Drama,Romance,War",7.5,"12,221",...,None,NaN,None,None,None,None,None,96.0,7.9,3035.0
1,2,7th Heaven,1927/28,Fox,Nominee,1927,110,"Drama,Romance",7.7,"3,439",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,The Racket,1927/28,The Caddo Company,Nominee,1928,84,"Crime,Drama,Film-Noir",6.7,"1,257",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,The Broadway Melody,1928/29,Metro-Goldwyn-Mayer,Winner,1929,100,"Drama,Musical,Romance",5.7,"6,890",...,None,NaN,MGM,Original Screenplay,Musical,None,None,38.0,6.7,2017.0
4,5,Alibi,1928/29,Feature Productions,Nominee,1929,91,"Action,Crime,Romance",5.8,765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Split the merged file into 4 and clean the missing values

### 1. Movies table - Select only relevant columns, check for null values and clean the missing data

In [8]:
movies_df = df_joined[['movie_id', 'Film', 'Year of Release', 'Original Release Date', 'Movie Time', 'Movie Genre',
                     'Content Rating', 'Film Studio/Producer(s)', 'Directors', 'Actors']]
movies_df.head()

,movie_id,Film,Year of Release,Original Release Date,Movie Time,Movie Genre,Content Rating,Film Studio/Producer(s),Directors,Actors
0,1,Wings,1927,1927-08-12,144,"Drama,Romance,War",PG-13,Famous Players-Lasky,William Wellman,"Clara Bow, Charles 'Buddy' Rogers, Richard Arl..."
1,2,7th Heaven,1927,NaN,110,"Drama,Romance",NaN,Fox,NaN,NaN
2,3,The Racket,1928,NaN,84,"Crime,Drama,Film-Noir",NaN,The Caddo Company,NaN,NaN
3,4,The Broadway Melody,1929,1929-02-01,100,"Drama,Musical,Romance",NR,Metro-Goldwyn-Mayer,Harry Beaumont,"Anita Page, Bessie Love, Charles King, Jed Pro..."
4,5,Alibi,1929,NaN,91,"Action,Crime,Romance",NaN,Feature Productions,NaN,NaN


In [9]:
# Check for null values
movies_df.isnull().sum()

movie_id                     0
Film                         0
Year of Release              0
Original Release Date      132
Movie Time                   0
Movie Genre                  0
Content Rating             132
Film Studio/Producer(s)      0
Directors                  134
Actors                     132
dtype: int64

In [10]:
movies_df['Content Rating'].value_counts()

R        138
NR       107
PG        78
PG-13     68
G         48
Name: Content Rating, dtype: int64

In [11]:
# Replace null values for "Content Rating" with None
movies_df['Content Rating'].fillna(value='None', inplace=True)

In [12]:
movies_df['Content Rating'].value_counts()

R        138
None     132
NR       107
PG        78
PG-13     68
G         48
Name: Content Rating, dtype: int64

In [13]:
# Replace all null values (except Original Release date) with spaces as fields are string
movies_df[['Directors', 'Actors']] = movies_df[['Directors', 'Actors']].fillna(' ')
movies_df.head(10)

,movie_id,Film,Year of Release,Original Release Date,Movie Time,Movie Genre,Content Rating,Film Studio/Producer(s),Directors,Actors
0,1,Wings,1927,1927-08-12,144,"Drama,Romance,War",PG-13,Famous Players-Lasky,William Wellman,"Clara Bow, Charles 'Buddy' Rogers, Richard Arl..."
1,2,7th Heaven,1927,NaN,110,"Drama,Romance",None,Fox,,
2,3,The Racket,1928,NaN,84,"Crime,Drama,Film-Noir",None,The Caddo Company,,
3,4,The Broadway Melody,1929,1929-02-01,100,"Drama,Musical,Romance",NR,Metro-Goldwyn-Mayer,Harry Beaumont,"Anita Page, Bessie Love, Charles King, Jed Pro..."
4,5,Alibi,1929,NaN,91,"Action,Crime,Romance",None,Feature Productions,,
5,6,Hollywood Revue,1929,NaN,130,"Comedy,Music",None,Metro-Goldwyn-Mayer,,
6,7,In Old Arizona,1928,1928-12-25,95,Western,NR,Fox,"Irving Cummings, Raoul Walsh","Warner Baxter, Edmund Lowe, Dorothy Burgess, J..."
7,8,The Patriot,1928,NaN,113,"Drama,History,Thriller",None,Paramount Famous Lasky,,
8,9,All Quiet on the Western Front,1930,NaN,152,"Drama,War",None,Universal,,
9,10,The Big House,1930,1930-05-14,87,"Crime,Drama,Thriller",NR,Cosmopolitan,George W. Hill,"Wallace Beery, Robert Montgomery, Chester Morr..."


In [14]:
# Recheck for null values
movies_df.isnull().sum()

movie_id                     0
Film                         0
Year of Release              0
Original Release Date      132
Movie Time                   0
Movie Genre                  0
Content Rating               0
Film Studio/Producer(s)      0
Directors                    0
Actors                       0
dtype: int64

In [15]:
# Rename DataFrame columns to match "movies" table
movies_df.columns = ['movie_id', 'movie_name', 'release_year', 'original_release_date',
                     'movie_length', 'genre', 'rating', 'movie_producer',
                     'movie_director', 'movie_actor']
movies_df.head()

,movie_id,movie_name,release_year,original_release_date,movie_length,genre,rating,movie_producer,movie_director,movie_actor
0,1,Wings,1927,1927-08-12,144,"Drama,Romance,War",PG-13,Famous Players-Lasky,William Wellman,"Clara Bow, Charles 'Buddy' Rogers, Richard Arl..."
1,2,7th Heaven,1927,NaN,110,"Drama,Romance",None,Fox,,
2,3,The Racket,1928,NaN,84,"Crime,Drama,Film-Noir",None,The Caddo Company,,
3,4,The Broadway Melody,1929,1929-02-01,100,"Drama,Musical,Romance",NR,Metro-Goldwyn-Mayer,Harry Beaumont,"Anita Page, Bessie Love, Charles King, Jed Pro..."
4,5,Alibi,1929,NaN,91,"Action,Crime,Romance",None,Feature Productions,,


### 2. Movie_Ratings table - select only required columns, check null values and clean the missing data 

In [16]:
movie_rating_df = df_joined[['movie_id', 'IMDB Rating_x', 'IMDB Votes_x', 'Tomatometer Rating',
                             'Tomatometer Count', 'Audience Rating', 'Audience Count']]
movie_rating_df.head()

,movie_id,IMDB Rating_x,IMDB Votes_x,Tomatometer Rating,Tomatometer Count,Audience Rating,Audience Count
0,1,7.5,"12,221",93.0,46.0,78.0,3530.0
1,2,7.7,"3,439",NaN,NaN,NaN,NaN
2,3,6.7,"1,257",NaN,NaN,NaN,NaN
3,4,5.7,"6,890",33.0,24.0,21.0,1813.0
4,5,5.8,765,NaN,NaN,NaN,NaN


In [17]:
# Check for null values
movie_rating_df.isnull().sum()

movie_id                0
IMDB Rating_x           0
IMDB Votes_x            0
Tomatometer Rating    132
Tomatometer Count     132
Audience Rating       132
Audience Count        132
dtype: int64

In [18]:
# Fill all null values with 0 as fields are numeric
movie_rating_df = movie_rating_df.fillna(0)
movie_rating_df.head()

,movie_id,IMDB Rating_x,IMDB Votes_x,Tomatometer Rating,Tomatometer Count,Audience Rating,Audience Count
0,1,7.5,"12,221",93.0,46.0,78.0,3530.0
1,2,7.7,"3,439",0.0,0.0,0.0,0.0
2,3,6.7,"1,257",0.0,0.0,0.0,0.0
3,4,5.7,"6,890",33.0,24.0,21.0,1813.0
4,5,5.8,765,0.0,0.0,0.0,0.0


In [19]:
# Recheck for null values
movie_rating_df.isnull().sum()

movie_id              0
IMDB Rating_x         0
IMDB Votes_x          0
Tomatometer Rating    0
Tomatometer Count     0
Audience Rating       0
Audience Count        0
dtype: int64

In [20]:
# Check the data types 
movie_rating_df.dtypes

movie_id                int64
IMDB Rating_x         float64
IMDB Votes_x           object
Tomatometer Rating    float64
Tomatometer Count     float64
Audience Rating       float64
Audience Count        float64
dtype: object

In [21]:
# Remove 'comma' from IMDB_Votes and convert to integer
movie_rating_df['IMDB Votes_x'] = movie_rating_df['IMDB Votes_x'].str.replace(',', '').astype(int)
movie_rating_df.head(10)

,movie_id,IMDB Rating_x,IMDB Votes_x,Tomatometer Rating,Tomatometer Count,Audience Rating,Audience Count
0,1,7.5,12221,93.0,46.0,78.0,3530.0
1,2,7.7,3439,0.0,0.0,0.0,0.0
2,3,6.7,1257,0.0,0.0,0.0,0.0
3,4,5.7,6890,33.0,24.0,21.0,1813.0
4,5,5.8,765,0.0,0.0,0.0,0.0
5,6,5.7,2004,0.0,0.0,0.0,0.0
6,7,5.6,1019,56.0,9.0,38.0,356.0
7,8,7.4,18,0.0,0.0,0.0,0.0
8,9,8.1,59214,0.0,0.0,0.0,0.0
9,10,7.1,2079,75.0,8.0,69.0,323.0


In [22]:
# Convert columns to integer
movie_rating_df[['Tomatometer Rating', 'Tomatometer Count', 'Audience Rating', 'Audience Count']] = movie_rating_df[['Tomatometer Rating', 'Tomatometer Count', 'Audience Rating', 'Audience Count']].astype(int)
movie_rating_df.head(10)

,movie_id,IMDB Rating_x,IMDB Votes_x,Tomatometer Rating,Tomatometer Count,Audience Rating,Audience Count
0,1,7.5,12221,93,46,78,3530
1,2,7.7,3439,0,0,0,0
2,3,6.7,1257,0,0,0,0
3,4,5.7,6890,33,24,21,1813
4,5,5.8,765,0,0,0,0
5,6,5.7,2004,0,0,0,0
6,7,5.6,1019,56,9,38,356
7,8,7.4,18,0,0,0,0
8,9,8.1,59214,0,0,0,0
9,10,7.1,2079,75,8,69,323


In [23]:
# Recheck the data types 
movie_rating_df.dtypes

movie_id                int64
IMDB Rating_x         float64
IMDB Votes_x            int32
Tomatometer Rating      int32
Tomatometer Count       int32
Audience Rating         int32
Audience Count          int32
dtype: object

In [24]:
# Rename DataFrame columns to match "movie_ratings" table
movie_rating_df.columns = ['movie_id', 'imdb_rating', 'imdb_votes', 'tomatometer_rating', 'tomatometer_count',
                          'audience_rating', 'audience_count']
movie_rating_df.head()

,movie_id,imdb_rating,imdb_votes,tomatometer_rating,tomatometer_count,audience_rating,audience_count
0,1,7.5,12221,93,46,78,3530
1,2,7.7,3439,0,0,0,0
2,3,6.7,1257,0,0,0,0
3,4,5.7,6890,33,24,21,1813
4,5,5.8,765,0,0,0,0


### 3. Oscar_Details table - select only relevant columns, check for null values and clean missing data

In [25]:
oscar_df = df_joined[['movie_id', 'Oscar Year', 'Award']]
oscar_df.head()

,movie_id,Oscar Year,Award
0,1,1927/28,Winner
1,2,1927/28,Nominee
2,3,1927/28,Nominee
3,4,1928/29,Winner
4,5,1928/29,Nominee


In [26]:
# Check for null values
oscar_df.isnull().sum()

movie_id      0
Oscar Year    0
Award         0
dtype: int64

In [27]:
# Format 'Oscar Year' to retain only first 4 characters
oscar_df['Oscar Year'] = oscar_df['Oscar Year'].str[:4]
oscar_df.head(10)

,movie_id,Oscar Year,Award
0,1,1927,Winner
1,2,1927,Nominee
2,3,1927,Nominee
3,4,1928,Winner
4,5,1928,Nominee
5,6,1928,Nominee
6,7,1928,Nominee
7,8,1928,Nominee
8,9,1929,Winner
9,10,1929,Nominee


In [28]:
# Rename DataFrame columns to match "oscar_details" table
oscar_df.columns = ['movie_id', 'oscar_year', 'award']
oscar_df.head()

,movie_id,oscar_year,award
0,1,1927,Winner
1,2,1927,Nominee
2,3,1927,Nominee
3,4,1928,Winner
4,5,1928,Nominee


### 4. Movie_Revenue table - select only relevant column, check for null values and clean up missing values. Load rows with atleast one financial data

In [29]:
movie_revenue_df = df_joined[['movie_id','US Gross', 'Worldwide Gross', 'Production Budget']]
movie_revenue_df.head()

,movie_id,US Gross,Worldwide Gross,Production Budget
0,1,NaN,NaN,2000000.0
1,2,NaN,NaN,NaN
2,3,NaN,NaN,NaN
3,4,2800000.0,4358000.0,379000.0
4,5,NaN,NaN,NaN


In [30]:
# Check for null values
movie_revenue_df.isnull().sum()

movie_id               0
US Gross             381
Worldwide Gross      381
Production Budget    380
dtype: int64

In [31]:
# Replace null values with 0 as columns are numeric
movie_revenue_df= movie_revenue_df.fillna(0)
movie_revenue_df.head()

,movie_id,US Gross,Worldwide Gross,Production Budget
0,1,0.0,0.0,2000000.0
1,2,0.0,0.0,0.0
2,3,0.0,0.0,0.0
3,4,2800000.0,4358000.0,379000.0
4,5,0.0,0.0,0.0


In [32]:
# Recheck for null values
movie_revenue_df.isnull().sum()

movie_id             0
US Gross             0
Worldwide Gross      0
Production Budget    0
dtype: int64

In [33]:
# Select only those rows that have atleast one financial data
clean_movie_rev = movie_revenue_df.loc[(movie_revenue_df['US Gross'] > 0) | (movie_revenue_df['Worldwide Gross'] > 0) | (movie_revenue_df['Production Budget'] > 0) ]
clean_movie_rev.head(10)

,movie_id,US Gross,Worldwide Gross,Production Budget
0,1,0.0,0.0,2000000.0
3,4,2800000.0,4358000.0,379000.0
7,8,113330342.0,215300000.0,110000000.0
15,16,15000000.0,15000000.0,4000000.0
27,28,2300000.0,2300000.0,439000.0
31,32,50003303.0,50003303.0,15000000.0
33,34,2000000.0,2000000.0,200000.0
35,36,3500000.0,3500000.0,4500000.0
36,37,2500000.0,2500000.0,325000.0
38,39,48000000.0,62000000.0,36000000.0


In [34]:
# Rename columns to match "movie_revenue" table
clean_movie_rev.columns = ['movie_id', 'us_gross_income', 'worldwide_gross_income', 'production_budget' ]
clean_movie_rev.head()

,movie_id,us_gross_income,worldwide_gross_income,production_budget
0,1,0.0,0.0,2000000.0
3,4,2800000.0,4358000.0,379000.0
7,8,113330342.0,215300000.0,110000000.0
15,16,15000000.0,15000000.0,4000000.0
27,28,2300000.0,2300000.0,439000.0


## Connect to PostGres and load tables

In [35]:
# Create the engine
engine = create_engine(f'postgresql://{user_key}@localhost:5432/Oscar-Winning-Predictions')

In [36]:
# Load the movies table
movies_df.to_sql(name='movies', con=engine, if_exists='append', index=False)

In [37]:
# Load the movie_ratings table
movie_rating_df.to_sql(name='movie_ratings', con=engine, if_exists='append', index=False)

In [38]:
# Load the oscar_details table
oscar_df.to_sql(name='oscar_details', con=engine, if_exists='append', index=False)

In [39]:
# Load the movie_revenue table
clean_movie_rev.to_sql(name='movie_revenue', con=engine, if_exists='append', index=False)